In [5]:
!pip install tensorflow 

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow_recommenders

# Simple two tower architecture

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1. **Dataset Creation**
# For demonstration, let's create a mock dataset of users and items.
num_users = 1000   # Total number of users
num_items = 5000   # Total number of items
num_samples = 10000  # Total number of interactions

# Randomly generating user-item interactions (user_id, item_id, interaction_score)
user_ids = np.random.randint(0, num_users, num_samples)
item_ids = np.random.randint(0, num_items, num_samples)
interaction_scores = np.random.randint(0, 2, num_samples)  # Binary interaction (0 or 1)

# 2. **Model Architecture**
# Input layers for users and items
user_input = layers.Input(shape=(1,), name='user_id')
item_input = layers.Input(shape=(1,), name='item_id')

# Embedding layers to convert user/item IDs into dense vectors
embedding_dim = 50  # Size of the embedding vectors
user_embedding = layers.Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
item_embedding = layers.Embedding(input_dim=num_items, output_dim=embedding_dim)(item_input)

# Flatten the embeddings to convert them from 2D to 1D
user_vec = layers.Flatten()(user_embedding)
item_vec = layers.Flatten()(item_embedding)

# User Tower: Define the neural network for user features
user_tower = layers.Dense(128, activation='relu')(user_vec)
user_tower = layers.Dense(64, activation='relu')(user_tower)

# Item Tower: Define the neural network for item features
item_tower = layers.Dense(128, activation='relu')(item_vec)
item_tower = layers.Dense(64, activation='relu')(item_tower)

# Combine the outputs of both towers
combined = layers.concatenate([user_tower, item_tower])

# Output layer: Final dense layer to predict the interaction score
output = layers.Dense(1, activation='sigmoid')(combined)

# Create the model
model = keras.Model(inputs=[user_input, item_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. **Model Summary**
model.summary()

# 4. **Train the Model**
# Split the data into training and testing sets
train_size = int(0.8 * num_samples)
user_train = user_ids[:train_size]
item_train = item_ids[:train_size]
y_train = interaction_scores[:train_size]

user_test = user_ids[train_size:]
item_test = item_ids[train_size:]
y_test = interaction_scores[train_size:]

# Fit the model
history = model.fit(
    [user_train, item_train],
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=([user_test, item_test], y_test)
)

# 5. **Evaluate the Model**
loss, accuracy = model.evaluate([user_test, item_test], y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 6. **Make Predictions**
# Example: Predict interaction score for a specific user-item pair
user_id_example = np.array([1])  # Example user ID
item_id_example = np.array([10])  # Example item ID
predicted_score = model.predict([user_id_example, item_id_example])
print(f"Predicted interaction score for user {user_id_example[0]} and item {item_id_example[0]}: {predicted_score[0][0]:.4f}")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_id (InputLayer)          │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ item_id (InputLayer)          │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 1, 50)             │          50,000 │ user_id[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, 1, 50)             │         250,000 │ item_id[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 50)                │               0 │ embedding_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │           6,528 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 128)               │           6,528 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           8,256 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 64)                │           8,256 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 128)               │               0 │ dense_1[0][0],             │
│                               │                           │                 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 1)                 │             129 │ concatenate[0][0]          │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 329,697 (1.26 MB)

 Trainable params: 329,697 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5003 - loss: 0.6938 - val_accuracy: 0.5090 - val_loss: 0.6932
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6840 - loss: 0.6394 - val_accuracy: 0.5010 - val_loss: 0.7358
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7820 - loss: 0.4701 - val_accuracy: 0.4995 - val_loss: 0.8549
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8137 - loss: 0.3895 - val_accuracy: 0.4935 - val_loss: 1.1402
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8326 - loss: 0.3357 - val_accuracy: 0.4990 - val_loss: 1.3282
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8404 - loss: 0.3189 - val_accuracy: 0.4960 - val_loss: 1.5475
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8437 - loss: 0.3091 - val_accuracy: 0.4950 - val_loss: 1.8447
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8506 - loss: 0.2950 - val_accuracy: 0.

In [4]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd

# Example dataset
data = {
    "user_id": ["user_1", "user_2", "user_3", "user_4"],
    "movie_title": ["Movie A", "Movie B", "Movie C", "Movie D"]
}

# Create DataFrames
df = pd.DataFrame(data)

# Convert DataFrame to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices({
    "user_id": df["user_id"].values,
    "movie_title": df["movie_title"].values
})
